In [1]:
# import moduel
import pandas as pd
import numpy as np
import os 
from os import listdir
import re

### 노원구

In [2]:
total_nowon = pd.DataFrame(columns=['미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
total_nowon

,미세먼지(PM-10),소음(db),온도(℃),습도(%),초미세먼지(PM-2.5),PM10단계,PM25단계,HDONG_NM


In [3]:
# 함수화
# 상계 2동
# 인자로 받을 값
# V10O1610616, V10O1610200, V10O1610312, V10O1611100, 상계 2동

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
V10O1610616 = pd.read_csv('{}.csv'.format('V10O1610616'))
V10O1610200 = pd.read_csv('{}.csv'.format('V10O1610200'))
V10O1610312 = pd.read_csv('{}.csv'.format('V10O1610312'))
V10O1611100 = pd.read_csv('{}.csv'.format('V10O1611100'))

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(V10O1610616)
df = df.append(V10O1610200)
df = df.append(V10O1610312)
df = df.append(V10O1611100)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계2동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
#     print(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계2동'
total_nowon = total_nowon.append(hour_df)
hour_df.head()

8760


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,미세먼지(PM-10),소음(db),온도(℃),습도(%),초미세먼지(PM-2.5),PM10단계,PM25단계,HDONG_NM
날짜,,,,,,,,
2018-04-01 00:00:00,66.0,51.0,15.0,51.0,38.0,보통,나쁨,상계2동
2018-04-01 01:00:00,66.0,51.0,14.0,51.0,37.0,보통,나쁨,상계2동
2018-04-01 02:00:00,67.0,50.0,14.0,52.0,33.0,보통,보통,상계2동
2018-04-01 03:00:00,65.0,50.0,14.0,53.0,32.0,보통,보통,상계2동
2018-04-01 04:00:00,64.0,50.0,14.0,53.0,37.0,보통,나쁨,상계2동


In [4]:
# 상계 6,7동
# V10O1610376, V10O1610351, V10O1611104

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
V10O1610376 = pd.read_csv('{}.csv'.format('V10O1610376'))
V10O1610351 = pd.read_csv('{}.csv'.format('V10O1610351'))
V10O1611104 = pd.read_csv('{}.csv'.format('V10O1611104'))

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(V10O1610376)
df = df.append(V10O1610351)
df = df.append(V10O1611104)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계6.7동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')


# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계6.7동'
total_nowon = total_nowon.append(hour_df)
hour_df.head()

Index(['날짜', 'HDONG_NM', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
       '초미세먼지(PM-2.5)'],
      dtype='object')
8760


,미세먼지(PM-10),소음(db),온도(℃),습도(%),초미세먼지(PM-2.5),PM10단계,PM25단계,HDONG_NM
날짜,,,,,,,,
2018-04-01 00:00:00,59.0,50.0,14.0,54.0,36.0,보통,나쁨,상계6.7동
2018-04-01 01:00:00,62.0,50.0,14.0,50.0,37.0,보통,나쁨,상계6.7동
2018-04-01 02:00:00,58.0,50.0,14.0,52.0,29.0,보통,보통,상계6.7동
2018-04-01 03:00:00,56.0,50.0,13.0,53.0,28.0,보통,보통,상계6.7동
2018-04-01 04:00:00,56.0,50.0,13.0,53.0,33.0,보통,보통,상계6.7동


In [5]:
# 하계1동 작업
# V10O1610610, V10O1610102

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
V10O1610610 = pd.read_csv('{}.csv'.format('V10O1610610'))
V10O1610102 = pd.read_csv('{}.csv'.format('V10O1610102'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(V10O1610610)
df = df.append(V10O1610102)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '하계1동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '하계1동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [6]:
# 공릉 2동
# V10O1610642, V10O1611652

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610642'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611652'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '공릉2동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '공릉2동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [7]:
# 상계 5동
# V10O1610297, V10O1611102

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610297'))
temp2 = pd.read_csv('{}.csv'.format('V10O1611102'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계5동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계5동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [8]:
# 공릉 1동
# V10O1610629, V10O1610630

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610629'))
temp2 = pd.read_csv('{}.csv'.format('V10O1610630'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)
df = df.append(temp2)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '공릉1동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '공릉1동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [9]:
# 상계 10동
# V10O1610293

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610293'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계10동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계10동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [10]:
# 상계 1동
# V10O1610356

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610356'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계1동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계1동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [11]:
# 월계 3동
# V10O1610643

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1610643'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '월계3동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '월계3동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [12]:
# 중계 2,3동
# V10O1611097

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1611097'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '중계2.3동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '중계2.3동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [13]:
# 중계 본동
# V10O1612126

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1612126'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '중계본동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '중계본동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [14]:
# 상계 3,4동
# V10O1611150

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1611150'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '상계3.4동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '상계3.4동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [21]:
# 월계 1동
# V10O1611229

# 데이터가 있는 폴더로 작업폴더 변경
data_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리"
out_dir = "C:\\Users\\User\\Documents\\data_weatherforcast\\노원구_전처리_개별동"
if not os.path.exists(out_dir): # 만약 경로가 없으면, 
    os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.
working_dir = os.chdir(data_dir)

# 데이터 읽기
temp1 = pd.read_csv('{}.csv'.format('V10O1611229'))

# 날짜와 column명만 있는 빈 데이터 프레임 생성
date_range = pd.date_range("2018-04-01-00","2019-03-31-23", freq="H")
# print(date_range)
for_fill = pd.DataFrame(index=date_range, columns=['미세먼지(PM-10)','소음(db)','온도(℃)','습도(%)',
                                                   '초미세먼지(PM-2.5)','PM10단계','PM25단계','HDONG_NM'])
# print(for_fill)

# 데이터 병합
df = pd.DataFrame(columns=['tm', '측정기 고유번호', '미세먼지(PM-10)', '소음(db)', '온도(℃)', '습도(%)',
                           '초미세먼지(PM-2.5)'])
df = df.append(temp1)

# 측정기 고유번호 변경
df['측정기 고유번호'] = '월계1동'
df = df.rename(columns={'측정기 고유번호': 'HDONG_NM', 'tm':'날짜'})
# print(df.columns)

# 그룹별 묶고, 라벨링
date_ls = list(df['날짜']) # 날짜 데이터를 list형태로 변경

# 시간별로 date를 절사. :13까지가 시간단위!
hour_ls = []
for date in date_ls:
    date = date[:13]
    date = date.replace(' ','-')
    date = date.replace('-','')
    hour_ls.append(date)
df['날짜'] = hour_ls

# 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y%m%d%H')

# 소수점 0자리로 절사
hour_df = round(df.groupby(df['날짜']).mean(),0)
# print(len(hour_df))

# 단계별로 labeling
hour_df['PM10단계'] = None
hour_df['PM25단계'] = None

good = hour_df[hour_df['미세먼지(PM-10)'] < 31.0].index
hour_df['PM10단계'].loc[good] = '좋음'
soso = hour_df[hour_df['미세먼지(PM-10)'] >= 31.0].index
hour_df['PM10단계'].loc[soso] = '보통'
bad = hour_df[hour_df['미세먼지(PM-10)'] >= 81.0].index
hour_df['PM10단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['미세먼지(PM-10)'] >= 151.0].index
hour_df['PM10단계'].loc[terrible] = '매우나쁨'

# PM 2.5
good = hour_df[hour_df['초미세먼지(PM-2.5)'] < 16.0].index
hour_df['PM25단계'].loc[good] = '좋음'
soso = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 16.0].index
hour_df['PM25단계'].loc[soso] = '보통'
bad = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 36.0].index
hour_df['PM25단계'].loc[bad] = '나쁨'
terrible = hour_df[hour_df['초미세먼지(PM-2.5)'] >= 76.0].index
hour_df['PM25단계'].loc[terrible] = '매우 나쁨'

hour_df['HDONG_NM'] = '월계1동'

final = pd.merge(hour_df, for_fill, how='right',
                 left_index=True, right_index=True, suffixes=('','_y'))
final = final.drop(columns=['미세먼지(PM-10)_y', '소음(db)_y', '온도(℃)_y', '습도(%)_y',
                            '초미세먼지(PM-2.5)_y', 'PM10단계_y', 'PM25단계_y', 'HDONG_NM_y'])
final['HDONG_NM'] = final['HDONG_NM'].fillna(method='ffill')

print(len(final))
total_nowon = total_nowon.append(final)

8760


In [22]:
final.head()

,미세먼지(PM-10),소음(db),온도(℃),습도(%),초미세먼지(PM-2.5),PM10단계,PM25단계,HDONG_NM,STD_YMD
2018-04-01 00:00:00,45.0,50.0,14.0,54.0,24.0,보통,보통,월계1동,NaN
2018-04-01 01:00:00,49.0,50.0,14.0,54.0,28.0,보통,보통,월계1동,NaN
2018-04-01 02:00:00,76.0,50.0,14.0,50.0,42.0,보통,나쁨,월계1동,NaN
2018-04-01 03:00:00,72.0,50.0,14.0,48.0,43.0,보통,나쁨,월계1동,NaN
2018-04-01 04:00:00,58.0,50.0,14.0,50.0,33.0,보통,보통,월계1동,NaN


In [17]:
len(total_nowon)

113880

In [18]:
total_nowon.to_csv('total_nowon.csv', encoding='cp949')

In [19]:
# 월계2동
# 무악동
# 삼청동
# 창신2동
# 상계8동
# 중계4동
# 중계1동
# 상계9동
# 하계2동